# Project: Enhancing Visual Reasoning in VQA Tasks through Interactive Learning

For my course project, I want to explore if interactive learning can improve spatial reasoning in Visual Question Answering (VQA) tasks.

## Background

From our readings and discussions so far, one of the key observations I've made is that current neural network models struggle with tasks requiring spatial reasoning or counting. They often rely on **pattern recognition** rather than true spatial understanding. Given this, I am somewhat skeptical of the research path based on the inherent, restrictive approach and am more drawn to exploring **interactive learning**. Can interactive learning bridge this gap, or bring more insights to traditional models?

## Available Resources

**Dataset**: CLEVR, a diagnostic dataset for testing spatial and logical reasoning. It also provides scene graphs as a foundation of interactive learning.  
**Existing Baselines**: Neural network-based VQA models (e.g., CNN + LSTM).

## Current Approaches

1. Implement a baseline VQA model to evaluate counting and spatial reasoning tasks. 
2. Introduce online learning approach, using given answers and programs to iteratively improve model predictions.
Compare model performance across these approaches and analyze their reasoning capabilities.

## Expected Results

Improved accuracy on counting and spatial reasoning tasks through interaction.
Insights into the potential interactive methods for improving static learning in VQA systems.

CLEVR paper: https://arxiv.org/pdf/1612.06890

dataset: https://github.com/facebookresearch/clevr-dataset-gen

In [1]:
import json
import torch
import torch.nn as nn
from pathlib import Path
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from torchvision import models, transforms
import re
from collections import Counter

## 0. Load Data
In preliminary practice, I control the length of program to limit the complexity of questions.

In [2]:
folder_path = "/srv/data/CLEVR_v1.0/"

In [3]:
class CLEVRDataset(Dataset):
    def __init__(self, corpus_dir, split='train', transform=None, max_program_length=None):
        """
        Initialize CLEVR dataset
        :param corpus_dir: root directory path of CLEVR dataset
        :param split: data split ('train', 'val', 'test')
        :param transform: image preprocessing transform
        :param max_program_length: maximum length of program, subset questions
        """
        self.path = Path(corpus_dir)
        self.split = split
        # self.mode = mode  # 'train' or 'test'
        # self.feedback_mode = feedback_mode  # 'none', 'answer', 'program'
        self.transform = transform 
        self.max_program_length = max_program_length

        questions_file = self.path / f'questions/CLEVR_{self.split}_questions.json'
        all_items = json.load(questions_file.open())['questions']

        # subset
        if self.max_program_length is not None:
            self.items = [
                item for item in all_items if len(item.get('program', [])) <= self.max_program_length
            ]
        else:
            self.items = all_items

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        item = self.items[idx]

        # load image
        img_path = self.path / 'images' / self.split / item['image_filename']
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        # basic data
        data = {
            'image': image, 
            'question': item['question'], 
            'answer': item.get('answer', '<NO_ANS>'), 
            'program': item.get('program', [])  # optional
        }

        # # add feedback 
        # if self.mode == 'train' and self.feedback_mode != 'none':
        #     if self.feedback_mode == 'answer':
        #         data['feedback'] = {'answer': item.get('answer', '<NO_ANS>')}
        #     elif self.feedback_mode == 'program':
        #         data['feedback'] = {'program': item.get('program', [])}

        return data

In [4]:
# # load scene graphs

# scene_path = folder_path + "/scenes/CLEVR_val_scenes.json"
# with open(scene_path, 'r') as f:
#     scenes = json.load(f)

# # look into the structure
# print(scenes['scenes'][0])


In [5]:
# # load questions

# questions_path = folder_path + "/questions/CLEVR_val_questions.json"
# with open(questions_path, 'r') as f:
#     questions = json.load(f)

# # # look into the structure
# # print(questions['questions'][0])

In [6]:
# # collect simple 'count' questions as a starting point
# def extract_count_questions(questions_data, max_steps=3):
#     count_questions = []
#     for question in questions_data['questions']:
#         program = question['program']
#         if any(step['function'] == 'count' for step in program) and len(program) <= max_steps:
#             count_questions.append({
#                 'question_index': question['question_index'],
#                 'image_index': question['image_index'],
#                 'image_filename': question['image_filename'],
#                 'question': question['question'],
#                 'answer': question['answer'],
#                 'program': question['program']
#             })
#     return count_questions

# count_questions = extract_count_questions(questions, max_steps=3)

In [7]:
# print(f"Total 'count' questions found: {len(count_questions)}")
# print("Sample 'count' question:", count_questions[:5])

In [8]:
# # iterate the problem and analyze its program
# for question in count_questions:
#     print("Question:", question['question'])
#     print("Answer:", question['answer'])
#     print("Program:")
    
#     for step in question['program']:
#         print(f"  Function: {step['function']}, Inputs: {step['inputs']}, Values: {step['value_inputs']}")
#     print("-" * 40)

In [9]:
# random.shuffle(count_questions)

# # Split data for initialization and testing
# train_questions = count_questions[:200]
# test_questions = count_questions[200:262]

## 1. Baseline VQA Model

In previous lecture, Bill introduced a base VQA model, which utilized LSTM model to process question and VGG16 to extract image features.

In [10]:
train_dataset = CLEVRDataset(folder_path, split='train', max_program_length=3)

In [11]:
# construct a global vocab
def load_all_questions(folder_path):
    all_items = []
    for split in ['train', 'val', 'test']:
        questions_file = Path(folder_path) / f'questions/CLEVR_{split}_questions.json'
        all_items.extend(json.load(questions_file.open())['questions'])
    return all_items

tokenize = lambda text: re.findall(r'\w+|\S', text.lower())
def build_global_vocab(folder_path):
    all_items = load_all_questions(folder_path)

    # debug
    for item in all_items:
        if 'answer' not in item:
            print("Item without 'answer' key:", item)
            break
            
    all_questions = [tokenize(item['question']) for item in all_items if 'question' in item]
    all_answers = [item.get('answer', '<NO_ANS>') for item in all_items]

    question_vocab = Counter([token for question in all_questions for token in question])
    answer_vocab = Counter(all_answers)

    itos_q = ['<PAD>', '<UNK>'] + [s[0] for s in question_vocab.most_common()]
    stoi_q = {s: i for i, s in enumerate(itos_q)}
    pad_idx, unk_idx = stoi_q['<PAD>'], stoi_q['<UNK>']

    itos_a = ['<NO_ANS>'] + [s[0] for s in answer_vocab.most_common()]
    stoi_a = {s: i for i, s in enumerate(itos_a)}

    return question_vocab, answer_vocab, stoi_q, itos_q, stoi_a, itos_a, pad_idx, unk_idx


In [12]:
question_vocab, answer_vocab, stoi_q, itos_q, stoi_a, itos_a, pad_idx, unk_idx = build_global_vocab(folder_path)

Item without 'answer' key: {'image_index': 0, 'split': 'test', 'image_filename': 'CLEVR_test_000000.png', 'question_index': 0, 'question': 'Is there anything else that is the same shape as the small brown matte object?'}


In [13]:
# question_vocab

In [47]:
# answer_vocab

Counter({'no': 174753,
         'yes': 168916,
         '<NO_ANS>': 149988,
         '1': 70220,
         '0': 59178,
         'small': 38426,
         'rubber': 38134,
         'metal': 38100,
         'large': 38016,
         '2': 36155,
         'sphere': 25491,
         'cube': 25476,
         'cylinder': 25458,
         '3': 18624,
         'purple': 9674,
         'yellow': 9671,
         'blue': 9596,
         'brown': 9576,
         'green': 9560,
         'gray': 9458,
         'cyan': 9446,
         'red': 9401,
         '4': 9016,
         '5': 4375,
         '6': 1999,
         '7': 802,
         '8': 307,
         '9': 125,
         '10': 27})

In [15]:
# class LSTMEncoder(nn.Module):

#     def __init__(self, n_tokens, hidden_size, n_layers, dropout=0.5):
#         super().__init__()
#         self.embedding = nn.Embedding(n_tokens, hidden_size)
#         dropout = dropout if n_layers > 1 else 0
#         self.lstm = nn.LSTM(hidden_size, hidden_size, n_layers, dropout=dropout)

#     def forward(self, x, x_lens):
#         x, hidden = self.lstm(self.embedding(x))
#         # take the final token hidden state for each item in batch
#         return x[torch.arange(x.size(0)),x_lens-1,:] 

# class VQAModel(nn.Module):

#     def __init__(self,
#                 num_answers,
#                 text_encoder, text_dim,
#                 image_encoder, image_dim):
#         super().__init__()
#         self.text_encoder = text_encoder
#         self.image_encoder = image_encoder
#         self.classifier = nn.Linear(text_dim + image_dim, num_answers)
#         self.softmax = nn.Softmax(dim=0)

#     def forward(self, text, text_lens, image):
#         x_text = text_encoder(text, text_lens)
#         x_image = image_encoder(image)
#         x = torch.concat((x_text, x_image), axis=1)
#         return self.softmax(self.classifier(x))

# image_encoder = nn.Sequential(
#     models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1).features,
#     nn.AdaptiveAvgPool2d((7, 7)),
#     nn.Flatten(),
#     nn.Linear(25088, 4096),
#     nn.ReLU(),
#     nn.Linear(4096, 512)
# )
                              
# text_encoder = LSTMEncoder(len(itos_q), 50, 3)
# model = VQAModel(len(itos_a), image_encoder, 512, text_encoder, 50)


In [41]:
image_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [17]:
# def collate_fn(samples):
#     tokenized_qs = [list(map(lambda x: stoi_q.get(x, unk_idx), s['question'])) for s in samples]
#     question_lens = torch.LongTensor([len(q) for q in tokenized_qs])
#     qs_maxlen = question_lens.max()
#     tokenized_qs = torch.LongTensor([q + [pad_idx] * (qs_maxlen - len(q)) for q in tokenized_qs])
#     images = torch.stack([image_transforms(s['image']) for s in samples])
#     answers = torch.LongTensor([stoi_a[s['answer']] for s in samples])
#     return tokenized_qs, question_lens, images, answers

# train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=collate_fn)

In [18]:
# def collate_fn(samples):
#     tokenized_qs = [list(map(lambda x: stoi_q.get(x, unk_idx), tokenize(s['question']))) for s in samples]
#     question_lens = torch.LongTensor([len(q) for q in tokenized_qs])
#     qs_maxlen = question_lens.max()
#     tokenized_qs = torch.LongTensor([q + [pad_idx] * (qs_maxlen - len(q)) for q in tokenized_qs])
#     images = torch.stack([image_transforms(s['image']) for s in samples])
#     answers = torch.LongTensor([stoi_a.get(s['answer'], -1) for s in samples])  # unknown -> -1
#     return tokenized_qs, question_lens, images, answers

# train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=collate_fn)

In [19]:
# !nvidia-smi

# !ps -aux | grep python

In [20]:
# from tqdm.notebook import tqdm

# num_epochs = 5
# learning_rate = 0.001
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(device)
# model.to(device)

# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)

# n_total_step = len(train_loader)
# for epoch in range(num_epochs):
#    for i, batch in enumerate(tqdm(train_loader)):
#        x_text, x_text_lens, x_img, y = (item.to(device) for item in batch)       
#        y_hat = model(x_text, x_text_lens, x_img)
#        n_corrects = (y_hat.argmax(axis=1)==y).sum().item()
#        loss_value = criterion(y_hat, y)
#        loss_value.backward()
#        optimizer.step()
#        optimizer.zero_grad()
#        if (i+1) % 250 == 0:
#           print(f'epoch {epoch+1}/{num_epochs}, step: {i+1}/{n_total_step}: loss = {loss_value:.5f}, acc = {100*(n_corrects/y.size(0)):.2f}%')
#    print()

In [49]:
val_dataset = CLEVRDataset(folder_path, split='val', max_program_length=3)
# test_dataset = CLEVRDataset(folder_path, split='val', max_program_length=3)
# test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=collate_fn)

In [22]:
# def calculate_accuracy(model, dataloader, device=device):
#     model.eval()  
#     correct = 0
#     total = 0

#     with torch.no_grad():
#         for batch in dataloader:
#             tokenized_qs, question_lens, images, answers = batch
            
#             tokenized_qs = tokenized_qs.to(device)
#             question_lens = question_lens.to(device)
#             images = images.to(device)
#             answers = answers.to(device)

#             outputs = model(tokenized_qs, question_lens, images)
#             _, predicted = torch.max(outputs, dim=1)

#             correct += (predicted == answers).sum().item()
#             total += answers.size(0)
            
#     accuracy = correct / total
#     return accuracy


In [23]:
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# model = model.to(device)

# test_accuracy = calculate_accuracy(model, test_loader, device=device)
# print(f"Test Accuracy: {test_accuracy * 100:.2f}%") # 7.46% -> 2.71%

## 2. Online Learning with Program

In [24]:
def build_program_vocab(train_dataset):
    programs = [item.get('program', []) for item in train_dataset]
    
    # extract all functions
    all_functions = [step['function'] for program in programs for step in program]
    function_vocab = Counter(all_functions)

    # build index mapping
    itos_p = ['<PAD>'] + [func for func, _ in function_vocab.most_common()]
    stoi_p = {func: idx for idx, func in enumerate(itos_p)}

    return stoi_p, itos_p


In [25]:
stoi_p, itos_p = build_program_vocab(train_dataset)

print("String to Index:", stoi_p)
print("Index to String:", itos_p)

String to Index: {'<PAD>': 0, 'scene': 1, 'count': 2, 'exist': 3, 'filter_color': 4, 'filter_shape': 5, 'filter_material': 6, 'filter_size': 7}
Index to String: ['<PAD>', 'scene', 'count', 'exist', 'filter_color', 'filter_shape', 'filter_material', 'filter_size']


In [26]:
# def collate_fn_1(samples):
#     tokenized_qs = [list(map(lambda x: stoi_q.get(x, unk_idx), tokenize(s['question']))) for s in samples]
#     question_lens = torch.LongTensor([len(q) for q in tokenized_qs])
#     qs_maxlen = question_lens.max()
#     tokenized_qs = torch.LongTensor([q + [pad_idx] * (qs_maxlen - len(q)) for q in tokenized_qs])
#     images = torch.stack([image_transforms(s['image']) for s in samples])
#     answers = torch.LongTensor([stoi_a.get(s['answer'], -1) for s in samples])  # unknown -> -1
   
#     programs = []
#     for s in samples:
#         program_indices = torch.LongTensor([
#             stoi_p.get(step['function'], -1) for step in s['program']  
#         ])
#         programs.append(program_indices)

#     # padding
#     max_program_length = max(len(p) for p in programs)
#     padded_programs = torch.stack([
#         torch.cat([p, torch.full((max_program_length - len(p),), pad_idx)]) for p in programs
#     ])
    
#     return tokenized_qs, question_lens, images, answers, padded_programs

# train_loader_1 = DataLoader(train_dataset, batch_size=16, collate_fn=collate_fn_1)

In [27]:
# class VQAModelWithProgram(nn.Module):
#     def __init__(self, num_answers, image_encoder, image_dim, text_encoder, text_dim, program_vocab_size, program_dim):
#         super().__init__()
#         self.image_encoder = image_encoder  
#         self.text_encoder = text_encoder    
#         self.program_encoder = nn.Embedding(program_vocab_size, program_dim)  
#         self.classifier = nn.Linear(image_dim + text_dim + program_dim, num_answers)
#         self.softmax = nn.Softmax(dim=0)

#     def forward(self, questions, question_lens, programs, images):

#         image_features = self.image_encoder(images)
#         question_features = self.text_encoder(questions, question_lens)
#         program_features = self.program_encoder(programs).mean(dim=1) 

#         combined_features = torch.cat((image_features, question_features, program_features), dim=1)

#         output = self.classifier(combined_features)
#         return self.softmax(output)


In [28]:
# import random

# def online_learning(model, dataloader, optimizer, criterion, num_iterations=100, device=device):
#     performance = [] 

#     # convert the dataloader into a list of batches
#     all_batches = list(dataloader)
#     random.shuffle(all_batches) 

#     for iteration, batch in enumerate(dataloader):
#         if iteration >= num_iterations:  # control the number of iterations
#             break

#         # train on the current batch
#         model.train()

#         model = model.to(device)
#         tokenized_qs, question_lens, images, answers, programs = batch
#         tokenized_qs = tokenized_qs.to(device)
#         question_lens = question_lens.to(device)
#         images = images.to(device)
#         answers = answers.to(device)
#         programs = programs.to(device)

#         outputs = model(tokenized_qs, question_lens, programs, images)
#         loss = criterion(outputs, answers)

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         # use remaining unseen batches for testing
#         unseen_batches = all_batches[iteration + 1:]
#         if unseen_batches:
#             test_batch = unseen_batches[0]
#             tokenized_qs, question_lens, images, answers, programs = test_batch
            
#             tokenized_qs = tokenized_qs.to(device)
#             question_lens = question_lens.to(device)
#             images = images.to(device)
#             answers = answers.to(device)
#             programs = programs.to(device)

#             model.eval()
#             with torch.no_grad():
#                 outputs = model(tokenized_qs, question_lens, programs, images)
#                 _, predicted = torch.max(outputs, dim=1)
#                 test_accuracy = (predicted == answers).float().mean().item()
#         else:
#             test_accuracy = 0.0  # no more unseen batches

#         print(f"Iteration {iteration+1}/{num_iterations}: Loss = {loss.item():.4f}, Test Accuracy = {test_accuracy * 100:.2f}%")

#     return performance


In [29]:
# model_withprogram = VQAModelWithProgram(len(itos_a), image_encoder, 512, text_encoder, 50, len(ftoi_p), 50)
# performance = online_learning(model_withprogram, train_loader_1, optimizer, criterion, num_iterations=25, device=device)

In [30]:
# # Initialize model with dynamic fusion
# model_with_dynamic_fusion = VQAModelWithDynamicFusion(
#     num_answers=len(itos_a),
#     image_encoder=image_encoder,
#     image_dim=512,
#     text_encoder=text_encoder,
#     text_dim=50,
#     program_vocab_size=len(itos_p),
#     program_dim=50,
#     hidden_dim=512
# )

# # Train with online learning
# performance = online_learning(
#     model_with_dynamic_fusion,
#     train_loader_1,
#     optimizer,
#     criterion,
#     num_iterations=25,
#     device=device
# )

The output indicates that the model's predictions are basically random..try to use dynamic fusion weights rather than simple concatenation? How to make the approach more "interactive"?

In [32]:
## Upgrade structure 
import torch
import torch.nn as nn
from torchvision import models
from transformers import BertModel, BertTokenizer
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Image Encoder：VGG16 -> ResNet
class ImageEncoder(nn.Module):
    def __init__(self, pretrained=True):
        super(ImageEncoder, self).__init__()
        resnet = models.resnet50(pretrained=pretrained)
        self.features = nn.Sequential(*list(resnet.children())[:-1])  # remove last layer
        self.proj = nn.Linear(2048, 512)  

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # flatten
        x = self.proj(x)
        return x

# Text Encoder: LSTM -> BERT
class TextEncoder(nn.Module):
    def __init__(self, pretrained_model_name='bert-base-uncased'):
        super(TextEncoder, self).__init__()
        self.bert = BertModel.from_pretrained(pretrained_model_name)
        self.proj = nn.Linear(768, 512)  

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # [CLS]
        x = self.proj(pooled_output)
        return x

# Program Encoder: LSTM
class ProgramEncoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=256):
        super(ProgramEncoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.proj = nn.Linear(hidden_dim, 512)  

    def forward(self, x):
        x = self.embedding(x)
        _, (h_n, _) = self.lstm(x)
        x = h_n[-1]  # last hidden state
        x = self.proj(x)
        return x

# Initialization
image_encoder = ImageEncoder()
text_encoder = TextEncoder()
program_encoder = ProgramEncoder(vocab_size=len(itos_p))  

In [33]:
## Define baseline model class (without program)
class BaselineVQAModel(nn.Module):
    def __init__(self, num_answers, image_encoder, text_encoder):
        super(BaselineVQAModel, self).__init__()
        self.image_encoder = image_encoder
        self.text_encoder = text_encoder
        self.classifier = nn.Linear(512 + 512, num_answers) 

    def forward(self, image, input_ids, attention_mask):
        image_features = self.image_encoder(image)
        text_features = self.text_encoder(input_ids, attention_mask)
        combined_features = torch.cat([image_features, text_features], dim=1)
        output = self.classifier(combined_features)
        return output

In [34]:
def train(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch in dataloader:
        images, input_ids, attention_mask, answers = batch
        images = images.to(device)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        answers = answers.to(device)

        optimizer.zero_grad()
        outputs = model(images, input_ids, attention_mask)
        loss = criterion(outputs, answers)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == answers).sum().item()
        total += answers.size(0)

    avg_loss = total_loss / len(dataloader)
    accuracy = correct / total
    return avg_loss, accuracy

In [35]:
def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in dataloader:
            images, input_ids, attention_mask, answers = batch 
            images = images.to(device)
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            answers = answers.to(device)

            outputs = model(images, input_ids, attention_mask)
            loss = criterion(outputs, answers)

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == answers).sum().item()
            total += answers.size(0)

    avg_loss = total_loss / len(dataloader)
    accuracy = correct / total
    return avg_loss, accuracy

In [36]:
from transformers import BertTokenizer

# Initialize a tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def collate_fn(samples):
    # extract necessary attributes
    questions = [s['question'] for s in samples]
    answers = [s['answer'] for s in samples]
    images = [s['image'] for s in samples]
    
    # Use BERT tokenizer to process questions
    tokenized = tokenizer(questions, padding=True, truncation=True, return_tensors='pt')
    input_ids = tokenized['input_ids']
    attention_mask = tokenized['attention_mask']

    # process images
    images = torch.stack([image_transforms(img) for img in images])

    # process answers
    answers = torch.tensor([stoi_a.get(ans, -1) for ans in answers])  # unknown -> -1

    return images, input_ids, attention_mask, answers

In [48]:
train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=16, collate_fn=collate_fn)

In [50]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

baseline_model = BaselineVQAModel(
    num_answers=len(itos_a),
    image_encoder=image_encoder,
    text_encoder=text_encoder
).to(device)

optimizer = AdamW(baseline_model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

num_epochs = 3
for epoch in range(num_epochs):
    train_loss, train_acc = train(baseline_model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(baseline_model, val_loader, criterion, device)
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc*100:.2f}%")

Epoch 1/3
Train Loss: 0.8594, Train Acc: 73.26%
Val Loss: 1.9207, Val Acc: 47.82%
Epoch 2/3
Train Loss: 0.4619, Train Acc: 83.75%
Val Loss: 2.3756, Val Acc: 45.15%
Epoch 3/3
Train Loss: 0.3264, Train Acc: 89.10%
Val Loss: 2.4332, Val Acc: 48.79%


In [51]:
# check
# train_answers = [item['answer'] for item in train_dataset]
# val_answers = [item['answer'] for item in val_dataset]

# print("Train answers distribution:", Counter(train_answers))
# print("Val answers distribution:", Counter(val_answers))

Train answers distribution: Counter({'yes': 764, '2': 308, '1': 273, '3': 207, '4': 128, 'no': 75, '5': 48, '6': 44, '0': 42, '7': 34, '8': 15, '9': 13, '10': 12})
Val answers distribution: Counter({'yes': 130, '2': 75, '3': 60, '1': 55, 'no': 20, '4': 19, '5': 16, '0': 13, '7': 10, '6': 8, '8': 3, '10': 2, '9': 1})


## WITH PROGRAM

In [81]:
# Dynamic Fusion: Multihead Attention
class DynamicFusion(nn.Module):
    def __init__(self, feature_dim=512, num_heads=8):
        super(DynamicFusion, self).__init__()
        self.multihead_attn = nn.MultiheadAttention(feature_dim, num_heads)
        self.proj = nn.Linear(feature_dim * 3, feature_dim) 

    def forward(self, image_features, text_features, program_features):
        # concat
        combined_features = torch.cat([image_features.unsqueeze(1),
                                       text_features.unsqueeze(1),
                                       program_features.unsqueeze(1)], dim=1)
        
        # multihead attention
        attn_output, _ = self.multihead_attn(combined_features, combined_features, combined_features)
        attn_output = attn_output.mean(dim=1)  # mean attention

        # fusion features
        fusion_features = self.proj(torch.cat([image_features, text_features, program_features], dim=1))
        return fusion_features



# Full Model
class VQAModelWithDynamicFusion(nn.Module):
    def __init__(self, num_answers, image_encoder, text_encoder, program_encoder, fusion_module):
        super(VQAModelWithDynamicFusion, self).__init__()
        self.image_encoder = image_encoder
        self.text_encoder = text_encoder
        self.program_encoder = program_encoder
        self.fusion_module = fusion_module
        self.classifier = nn.Linear(512, num_answers)  

    def forward(self, image, input_ids, attention_mask, program):
        image_features = self.image_encoder(image)
        text_features = self.text_encoder(input_ids, attention_mask)
        program_features = self.program_encoder(program)
        fusion_features = self.fusion_module(image_features, text_features, program_features)
        output = self.classifier(fusion_features)

        return output

fusion_module = DynamicFusion()
model = VQAModelWithDynamicFusion(
    num_answers=len(itos_a), 
    image_encoder=image_encoder,
    text_encoder=text_encoder,
    program_encoder=program_encoder,
    fusion_module=fusion_module
)


In [53]:
def collate_fn_1(samples):
    # extract necessary attributes
    questions = [s['question'] for s in samples]
    answers = [s['answer'] for s in samples]
    programs = [s['program'] for s in samples]  
    images = [s['image'] for s in samples]
    
    # Use BERT tokenizer to process questions
    tokenized = tokenizer(questions, padding=True, truncation=True, return_tensors='pt')
    input_ids = tokenized['input_ids']
    attention_mask = tokenized['attention_mask']

    # process images
    images = torch.stack([image_transforms(img) for img in images])

    # process programs
    max_program_length = max(max(len(p) for p in programs), 1)  
    padded_programs = []
    for program in programs:
        if len(program) == 0:
            program_indices = [0]  # index of <PAD> 
        else:
            # function -> index
            program_indices = [stoi_p.get(step['function'], 0) for step in program]
        # padding
        padded_program = program_indices + [0] * (max_program_length - len(program_indices))
        padded_programs.append(torch.tensor(padded_program, dtype=torch.long))
    padded_programs = torch.stack(padded_programs)

    # process answers
    answers = torch.tensor([stoi_a.get(ans, -1) for ans in answers])  # unknown -> -1

    return images, input_ids, attention_mask, padded_programs, answers

In [61]:
train_loader_1 = DataLoader(train_dataset, batch_size=16, collate_fn=collate_fn_1)
val_loader_1 = DataLoader(val_dataset, batch_size=16, collate_fn=collate_fn_1)

In [79]:
def train_1(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch in dataloader:
        images, input_ids, attention_mask, programs, answers = batch
        images = images.to(device)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        programs = programs.to(device)
        answers = answers.to(device)

        optimizer.zero_grad()
        outputs = model(images, input_ids, attention_mask, programs)
        loss = criterion(outputs, answers)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == answers).sum().item()
        total += answers.size(0)

    avg_loss = total_loss / len(dataloader)
    accuracy = correct / total
    return avg_loss, accuracy

def evaluate_1(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in dataloader:
            images, input_ids, attention_mask, programs, answers = batch
            images = images.to(device)
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            programs = programs.to(device)
            answers = answers.to(device)

            outputs = model(images, input_ids, attention_mask, programs)
            loss = criterion(outputs, answers)

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == answers).sum().item()
            total += answers.size(0)

    avg_loss = total_loss / len(dataloader)
    accuracy = correct / total
    return avg_loss, accuracy

In [83]:
num_epochs = 3
model = model.to(device)
criterion = nn.CrossEntropyLoss()

optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

for epoch in range(num_epochs):
    train_loss, train_acc = train_1(model, train_loader_1, optimizer, criterion, device)
    val_loss, val_acc = evaluate_1(model, val_loader_1, criterion, device)
    scheduler.step(val_loss)  # Adjust the learning rate based on the validation set loss

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc*100:.2f}%")

Epoch 1/3
Train Loss: 0.2763, Train Acc: 91.65%
Val Loss: 3.6969, Val Acc: 43.45%
Epoch 2/3
Train Loss: 0.1345, Train Acc: 95.77%
Val Loss: 3.4672, Val Acc: 47.57%
Epoch 3/3
Train Loss: 0.1020, Train Acc: 97.10%
Val Loss: 3.4026, Val Acc: 43.20%


In [77]:
import random

def online_learning(model, dataloader, optimizer, criterion, num_iterations=100, device=device):
    performance = []

    all_batches = list(dataloader)
    random.shuffle(all_batches)

    for iteration in range(num_iterations):
        # current batch
        batch = all_batches[iteration % len(all_batches)]

        model.train()
        model.to(device)

        images, input_ids, attention_mask, programs, answers = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        images = images.to(device)
        answers = answers.to(device)
        programs = programs.to(device)

        outputs = model(images, input_ids, attention_mask, programs)
        loss = criterion(outputs, answers)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # use remaining unseen batches for testing
        unseen_batches = all_batches[iteration + 1:]
        if unseen_batches:
            test_batch = unseen_batches[0]
            images_test, input_ids_test, attention_mask_test, programs_test, answers_test = test_batch

            input_ids_test = input_ids_test.to(device)
            attention_mask_test = attention_mask_test.to(device)
            images_test = images_test.to(device)
            answers_test = answers_test.to(device)
            programs_test = programs_test.to(device)

            model.eval()
            with torch.no_grad():
                outputs_test = model(images_test, input_ids_test, attention_mask_test, programs_test)
                _, predicted = torch.max(outputs_test, dim=1)
                test_accuracy = (predicted == answers_test).float().mean().item()
        else:
            test_accuracy = 0.0  

        print(f"Iteration {iteration + 1}/{num_iterations}: Loss = {loss.item():.4f}, Test Accuracy = {test_accuracy * 100:.2f}%")
        performance.append((loss.item(), test_accuracy))

    return performance

In [82]:
performance = online_learning(model, val_loader_1, optimizer, criterion, num_iterations=25, device=device)

Iteration 1/25: Loss = 3.7145, Test Accuracy = 0.00%
Iteration 2/25: Loss = 3.5823, Test Accuracy = 0.00%
Iteration 3/25: Loss = 3.4050, Test Accuracy = 8.33%
Iteration 4/25: Loss = 3.2283, Test Accuracy = 37.50%
Iteration 5/25: Loss = 2.8971, Test Accuracy = 50.00%
Iteration 6/25: Loss = 2.6252, Test Accuracy = 43.75%
Iteration 7/25: Loss = 2.6083, Test Accuracy = 37.50%
Iteration 8/25: Loss = 2.4597, Test Accuracy = 37.50%
Iteration 9/25: Loss = 2.4227, Test Accuracy = 25.00%
Iteration 10/25: Loss = 2.2773, Test Accuracy = 31.25%
Iteration 11/25: Loss = 2.3994, Test Accuracy = 43.75%
Iteration 12/25: Loss = 2.4256, Test Accuracy = 43.75%
Iteration 13/25: Loss = 1.9384, Test Accuracy = 50.00%
Iteration 14/25: Loss = 1.9498, Test Accuracy = 43.75%
Iteration 15/25: Loss = 1.4540, Test Accuracy = 43.75%
Iteration 16/25: Loss = 1.6086, Test Accuracy = 62.50%
Iteration 17/25: Loss = 1.9694, Test Accuracy = 31.25%
Iteration 18/25: Loss = 1.9138, Test Accuracy = 62.50%
Iteration 19/25: Loss 